In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [21]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [22]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [23]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110570  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,3.518186e+06
2018-02-28,3.169060e+06
2018-03-31,3.423607e+06
2018-04-30,3.287167e+06
2018-05-31,3.334496e+06
2018-06-30,3.182081e+06
2018-07-31,3.433448e+06
2018-08-31,3.353313e+06
2018-09-30,3.200192e+06


In [24]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    3.518186e+06
 2018-02-28    3.169060e+06
 2018-03-31    3.423607e+06
 2018-04-30    3.287167e+06
 2018-05-31    3.334496e+06
 2018-06-30    3.182081e+06
 2018-07-31    3.433448e+06
 2018-08-31    3.353313e+06
 2018-09-30    3.200192e+06
 2018-10-31    3.209108e+06
 2018-11-30    3.146478e+06
 2018-12-31    3.332365e+06
 2019-01-31    3.280900e+06
 2019-02-28    3.020861e+06
 2019-03-31    3.367239e+06
 2019-04-30    3.131234e+06
 2019-05-31    3.232367e+06
 2019-06-30    3.103971e+06
 2019-07-31    3.345175e+06
 2019-08-31    3.256620e+06
 2019-09-30    3.318130e+06
 2019-10-31    3.468178e+06
 2019-11-30    3.313089e+06
 2019-12-31    3.462946e+06
 2020-01-31    3.579045e+06
 2020-02-29    3.113281e+06
 2020-03-31    2.892804e+06
 2020-04-30    2.803007e+06
 2020-05-31    2.999872e+06
 2020-06-30    3.053327e+06
 2020-07-31    3.214149e+06
 2020-08-31    3.157210e+06
 2020-09-30    3.006232e+06
 2020-10-31    3.120614e+06
 2020-11-30    3.265163e+06

# ARIMA

In [25]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [26]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.063969
p-value : 0.001110
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [27]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1115.213, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=1111.540, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=1112.306, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=1113.712, Time=0.08 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=1116.499, Time=0.11 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=1117.268, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=1110.767, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=1110.533, Time=0.09 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=1110.014, Time=0.25 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.40 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.44 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=1111.621, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=inf, Time=0.30 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.36 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.51 sec
 ARIMA(3,1,0)(0,0,0)[1] intercept  

ARIMA(order=(1, 1, 2), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [29]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(1,1,1))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(1, 1, 1)   Log Likelihood                -548.094
Method:                       css-mle   S.D. of innovations         148864.663
Date:                Sat, 11 Sep 2021   AIC                           1104.188
Time:                        02:18:16   BIC                           1111.043
Sample:                             1   HQIC                          1106.684
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -8001.1207   2614.016     -3.061      0.002   -1.31e+04   -2877.743
ar.L1.D.y      0.2944      0.154      1.912      0.056      -0.007       0.596
ma.L1.D.y     -1.0000      0.068    -14.672      0.0

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [30]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([3025109.01352539, 3019715.09940535]), array([148864.66259168, 155183.75786984]), array([[2733339.636275  , 3316878.39077578],
       [2715560.52299488, 3323869.67581582]]))
